## Hints to "How He Did That"
He wrote a lot about specifically what to do in [this book](https://drive.google.com/file/d/1l5324Nlqy_5r4y9UWtDCywMCCQLsZwfr/view?usp=sharing), (PDF pg. 6-8)

He also cites [this paper](https://drive.google.com/file/d/1REwUE5Uvv-ArSxuzUMIXj4EkB0-kovIs/view?usp=sharing) for how to implement the excitation function.

In [20]:
%matplotlib inline
from audiolazy import *
from librosa import load
from librosa.core.pitch import piptrack
import librosa
# from aubio import miditofreq
import matplotlib.pyplot as plt
import math
import numpy as np

# Bless https://ptolemy.berkeley.edu/eecs20/sidebars/hertz/index.html
def freq_to_rad(freq, sr):
    return (freq / sr) * (2 * pi)

def pulse_train(freq, sr, samples):
    period = 1 / freq # period in seconds
    dur = period * sr
    pulse_period = impulse(dur=dur)

    i = 0
    out = Stream([])
    while i < samples:
        if samples - i < dur:
            out.append(pulse_period.peek(samples - i))
            i = samples
        else:
            out.append(pulse_period.peek(dur))
            i += dur
            
    return out

def excitation(rad, samples):
    out = []
    
    if rad == 0.0:
        return white_noise(dur=samples).take(samples)
    
    N = int(pi // rad)
    
    d_i = 0
    for i in range(samples):
        denominator = sin_table[d_i]
        
        n_i = (((2 * N) + 1) * d_i) % DEFAULT_TABLE_SIZE
        numerator = sin_table[n_i]
        
        new_d_i = d_i + (((rad / 2) * DEFAULT_TABLE_SIZE) / (2 * pi))
        d_i = int(new_d_i) % DEFAULT_TABLE_SIZE
        
        if denominator == 0:
            out.append(1)
        else:
            out.append(numerator / denominator)
            
    return out

def get_fundamental(freqs, mags, t):
    if not t < freqs.shape[1]:
        return 0
    return freqs[mags[:, t].argmax(), t]

## Analysis

In [21]:
sample = WavStream("nuttiness.wav")
y, sr = load("nuttiness.wav", sr=sample.rate)

BLK_SIZE = 441
freqs, mags = piptrack(y=y, sr=sr, fmin=50, fmax=300, hop_length=BLK_SIZE // 2)

original = []
coeffs = []
resids = []
resids_rms = []

sample = list(sample)
sample_len = len(sample) 
num_blocks = 0
i = 0

while i < sample_len:
    if sample_len - i > BLK_SIZE:
        blk = sample[i:i + BLK_SIZE]
    else:
        blk = sample[i:]
        
    try:
        analysis_filt = lpc.ncovar(blk, 5)
    except:
        analysis_filt = lpc.covar(blk, 5)

    coeffs.append(analysis_filt)
    residual = analysis_filt(blk)
    resids.append(residual.peek(len(blk)))
    resids_rms.append(list(envelope(Stream(residual.peek(len(blk))))))
    synth_filt = 1 / analysis_filt
    amplified_blk = synth_filt(Stream(residual))
    original += amplified_blk.peek(len(blk))
        
    i += len(blk)
    num_blocks += 1

with AudioIO(True) as player: # True means "wait for all sounds to stop"
    player.play(original, rate=sr, channels=2)

## Synthesis
TODO: add a way to condition time stretch

In [54]:
miditofreq = lambda m: 2**((m-69)/12.)*440
new_f0 = miditofreq(65)
stretchFactor = 2
mult = lambda length: int(length*stretchFactor)
def stretch(residBlock):
    res_arr = np.array(residBlock)
#     str_arr = librosa.effects.time_stretch(res_arr, stretchFactor)
    str_arr = librosa.core.resample(res_arr, sr, sr*stretchFactor)
    return str_arr
excit = Stream(excitation(freq_to_rad(new_f0, sr), mult(len(original)) ))

final = []

i = 0
while i < num_blocks - 1:        
    filt = coeffs[i]
    synth_filt = 1 / filt
    excitation_pulse = excit.take(mult(BLK_SIZE))
    stretched_resid = stretch(resids[i])
#     if len(stretched_resid) != len(excitation_pulse):
    print("wat", len(resids[i]), len(stretched_resid), len(excitation_pulse))
    excitation_pulse = [((0.05 * ex) + (0.95 * r)) for ex, r in zip(excitation_pulse, stretched_resid)]
    excitation_signal = [pulse * rms for pulse, rms in zip(excitation_pulse, stretch(resids_rms[i]))]
    amplified_blk = list(synth_filt(excitation_signal))
    final += amplified_blk
    
    i += 1

print(len(final))
    
with AudioIO(True) as player: # True means "wait for all sounds to stop"
    player.play(final, rate=sr, channels=2)

('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441, 882, 882)
('wat', 441